In [1]:
%load_ext autoreload
%autoreload 2

In [17]:
import pandas as pd
import random
import multiprocessing

from itertools import product
from datetime import date
from path import Path

import os
os.environ['NUMEXPR_MAX_THREADS'] = '32'

from mlbt.multiprocess import process_jobs
from mlbt.run_bt import run_bt
from mlbt.feature_eng import define_feature_configs
from mlbt.load_data import LENGTH_RANKING

DATA_DIR = Path("~/Dropbox/algotrading/data").expanduser()
s_csv = pd.read_csv(DATA_DIR / "symbols.csv", index_col="iqsymbol")

# Code to execute the pipeline inside a multiprocessing.Pool
# This creates the payloads that "run_frontend_payloads.ipynb" can use to generate the
# frontend payloads (.json loaded by the frontend)
ranking = ["equity_index", ""]


symbol_groups = list(
    {
        "agriculture": "Agriculture",
        "currency": "Currency",
        "energy": "Energy",
        "equity_index": "Equity Index",
        "interest_rate": "Interest Rate",
        "metals": "Metals",
    }.keys()
)

features = define_feature_configs()
print(len(features))
c_s = (len(features) // 32) + 1 
c_s = 1
print('c_s', c_s)
data_sim = {
    "start_date": [date(2000, 1, 1)],
    "end_date": [date(2021, 1, 1)],
    "symbol_groups": [symbol_groups],
#     "symbols": [['@NQ#C']],
#     "symbols": [[x] for x in LENGTH_RANKING],
    "bar_type": ["dollar"],
    "binarize": ["fixed_horizon"],
#     "binarize_params": [6, 12],
#     "binarize_params": [6],
    "binarize_params": [6, 12, 50, 200, 400],
#     "binarize_params": [6, 25, 50],
    "binarize_params": [25],
#     "binarize_params": [800],
    "alpha": ["none"],
#     "alpha": ["ma-cross_100_1000"],
#     "features": [features[i:i+c_s] for i in range(0, len(features), c_s)],
    "features": [features],
    "classifier": ["random_forest", "xgboost", "lgbm", "dummy"],
    "classifier": ["xgboost"],
#     "classifier": ["dummy"],
#     "classifier": ["lgbm"],
#     "classifier": ["svc"],
#     "classifier": ["knn"],
#     "classifier": ["xgboost", "lgbm", "dummy"],
#     "classifier": ["random_forest", "xgboost", "dummy"],
#     "classifier": ["tpot"],
#     "classifier": ["lgbm"],
    "num_threads": [32], # how many parallel hyperopti 
    "n_jobs": [32], # how many parallel infer
    "num_threads": [8], # how many parallel hyperopti 
    "n_jobs": [8], # how many parallel infer
#     "feat_imp_n_estimators": [1000],
#     "feat_imp_cv": [10],
    "func": [run_bt],
#     "feature_calc_only": [True],
#     "feature_imp_only": [True],
#     "check_completed": [True],
    "optimize_hypers": [False],
#     "reuse_hypers": [False],
#     "load_from_disk": [False],
    "skip_feature_imp": [True],
#     "pca_transform": [False],
#     "standard_scale": [False],
#     "hypers_n_iter": [10],
}

33
c_s 1


In [18]:
datas = list(dict(zip(data_sim.keys(), x)) for x in product(*data_sim.values()))
datas
# for data in datas:
#     data['features'] = list(sorted(data['features'], key=lambda x:random.random()))


[{'start_date': datetime.date(2000, 1, 1),
  'end_date': datetime.date(2021, 1, 1),
  'symbol_groups': ['agriculture',
   'currency',
   'energy',
   'equity_index',
   'interest_rate',
   'metals'],
  'bar_type': 'dollar',
  'binarize': 'fixed_horizon',
  'binarize_params': 25,
  'alpha': 'none',
  'features': [{'name': 'log_ret'},
   {'name': 'close', 'symbol': 'VIX.XO'},
   {'name': 'ffd', 'd': 0.5},
   {'name': 'roll', 'window': 10},
   {'name': 'rollimp', 'window': 10},
   {'name': 'amihud', 'window': 10},
   {'name': 'kyle', 'window': 10},
   {'name': 'ewm_mean', 'window': 10, 'symbol': {'name': 'volratio'}},
   {'name': 'roll', 'window': 25},
   {'name': 'rollimp', 'window': 25},
   {'name': 'amihud', 'window': 25},
   {'name': 'kyle', 'window': 25},
   {'name': 'ewm_mean', 'window': 25, 'symbol': {'name': 'volratio'}},
   {'name': 'roll', 'window': 50},
   {'name': 'rollimp', 'window': 50},
   {'name': 'amihud', 'window': 50},
   {'name': 'kyle', 'window': 50},
   {'name': 'ewm

In [19]:
len(datas)

1

In [20]:
# [x['features'][0] for x in datas]

In [21]:
num_threads = max(1, min(multiprocessing.cpu_count(), len(datas)) - 0)
num_threads
      

1

In [22]:
# res = process_jobs(datas, num_threads=num_threads)

In [23]:
for data in datas:
    run_bt(**data)

2020-02-24 12:07:13,308 config: {'DATA_DIR': Path('/Users/doda/Dropbox/algotrading/data'), 'F_PAYLOAD_DIR': Path('/Users/doda/pr/fincl/frontend/public/payloads'), 'start_date': datetime.date(2000, 1, 1), 'end_date': datetime.date(2021, 1, 1), 'data_freq': 'minutely', 'downsampling': 'cusum', 'vol_estimate': 100, 'symbol_groups': ['agriculture', 'currency', 'energy', 'equity_index', 'interest_rate', 'metals'], 'features': [{'name': 'log_ret'}, {'name': 'close', 'symbol': 'VIX.XO'}, {'name': 'ffd', 'd': 0.5}, {'name': 'roll', 'window': 10}, {'name': 'rollimp', 'window': 10}, {'name': 'amihud', 'window': 10}, {'name': 'kyle', 'window': 10}, {'name': 'ewm_mean', 'window': 10, 'symbol': {'name': 'volratio'}}, {'name': 'roll', 'window': 25}, {'name': 'rollimp', 'window': 25}, {'name': 'amihud', 'window': 25}, {'name': 'kyle', 'window': 25}, {'name': 'ewm_mean', 'window': 25, 'symbol': {'name': 'volratio'}}, {'name': 'roll', 'window': 50}, {'name': 'rollimp', 'window': 50}, {'name': 'amihud',

2020-02-24 12:07:24,148 XG#C: Feature engineering for 33 features
2020-02-24 12:07:24,754 Joined 33 features into (71217, 33) shape
2020-02-24 12:07:24,755 EX#C: Feature engineering for 33 features
2020-02-24 12:07:25,395 Joined 33 features into (74018, 33) shape
2020-02-24 12:07:25,396 @YM#C: Feature engineering for 33 features
2020-02-24 12:07:25,998 Joined 33 features into (39311, 33) shape
2020-02-24 12:07:25,999 @BO#C: Feature engineering for 33 features
2020-02-24 12:07:26,633 Joined 33 features into (93715, 33) shape
2020-02-24 12:07:26,634 @C#C: Feature engineering for 33 features
2020-02-24 12:07:27,269 Joined 33 features into (80542, 33) shape
2020-02-24 12:07:27,270 @FV#C: Feature engineering for 33 features
2020-02-24 12:07:27,929 Joined 33 features into (53497, 33) shape
2020-02-24 12:07:27,930 @TY#C: Feature engineering for 33 features
2020-02-24 12:07:28,533 Joined 33 features into (55083, 33) shape
2020-02-24 12:07:28,534 @TU#C: Feature engineering for 33 features
2020-

2020-02-24 12:07:51,351 BD#C: Get bins and feature imps
2020-02-24 12:07:51,980 Dropped label 0.0 0.005933050212340744
2020-02-24 12:07:52,078 bars (68392, 9), events (32049, 2), feats (68392, 33), bins (31834, 3), X_all (29416, 33), X_train (14708, 33)
2020-02-24 12:07:52,079 EZ#C: Get bins and feature imps
2020-02-24 12:07:53,127 Dropped label 0.0 0.023500222362661798
2020-02-24 12:07:53,245 bars (110422, 9), events (42748, 2), feats (110422, 33), bins (41719, 3), X_all (36374, 33), X_train (18187, 33)
2020-02-24 12:07:53,246 XG#C: Get bins and feature imps
2020-02-24 12:07:53,994 Dropped label 0.0 0.0016221070011296817
2020-02-24 12:07:54,098 bars (71217, 9), events (34548, 2), feats (71217, 33), bins (34467, 3), X_all (31713, 33), X_train (15856, 33)
2020-02-24 12:07:54,099 EX#C: Get bins and feature imps
2020-02-24 12:07:54,891 Dropped label 0.0 0.007075745273170166
2020-02-24 12:07:55,009 bars (74018, 9), events (34509, 2), feats (74018, 33), bins (34240, 3), X_all (31797, 33), X

2020-02-24 12:08:17,659 Dropped label 0.0 0.0022977022977022977
2020-02-24 12:08:17,724 bars (67132, 9), events (10035, 2), feats (67132, 33), bins (9987, 3), X_all (9643, 33), X_train (4821, 33)
2020-02-24 12:08:17,725 QSI#C: Get bins and feature imps
2020-02-24 12:08:18,368 Dropped label 0.0 0.003987399816579609
2020-02-24 12:08:18,464 bars (59640, 9), events (25104, 2), feats (59640, 33), bins (24979, 3), X_all (24544, 33), X_train (12272, 33)
2020-02-24 12:08:18,468 @AD#C [(897, 5), (897, 33), (897,), (898, 5), (898, 33), (898,)]
2020-02-24 12:08:18,469 @BP#C [(2532, 5), (2532, 33), (2532,), (2532, 5), (2532, 33), (2532,)]
2020-02-24 12:08:18,470 @CD#C [(1133, 5), (1133, 33), (1133,), (1134, 5), (1134, 33), (1134,)]
2020-02-24 12:08:18,471 @ED#C [(6708, 5), (6708, 33), (6708,), (6708, 5), (6708, 33), (6708,)]
2020-02-24 12:08:18,471 @ES#C [(12939, 5), (12939, 33), (12939,), (12940, 5), (12940, 33), (12940,)]
2020-02-24 12:08:18,472 @NQ#C [(7130, 5), (7130, 33), (7130,), (7130, 5), 

In [24]:
%debug

> /Users/doda/anaconda3/envs/metal/lib/python3.6/threading.py(295)wait()
    293         try:    # restore state no matter what (e.g., KeyboardInterrupt)
    294             if timeout is None:
--> 295                 waiter.acquire()
    296                 gotit = True
    297             else:

ipdb> exit
